In [3]:
import requests
from datetime import datetime
import psycopg2 as psql
from dotenv import load_dotenv, dotenv_values
import os

In [9]:
load_dotenv()

True

In [69]:
responce = requests.get('https://api.opendota.com/api/publicMatches?min_rank=40&max_rank=45')

In [70]:
responce

<Response [200]>

In [73]:
matches = responce.json()

In [75]:
len(matches)

100

In [77]:
matches[0].keys()

dict_keys(['match_id', 'match_seq_num', 'radiant_win', 'start_time', 'duration', 'lobby_type', 'game_mode', 'avg_rank_tier', 'num_rank_tier', 'cluster', 'radiant_team', 'dire_team'])

In [79]:
matches[0]['match_id']

7817126314

In [81]:
print(datetime.utcfromtimestamp(matches[0]['start_time']).strftime('%Y-%m-%d %H:%M:%S'))

2024-06-27 07:16:33


In [83]:
matches[3]['game_mode']

3

In [85]:
matches = [match for match in matches if match['game_mode']==22]

In [87]:
len(matches)

94

This method allows direct query of the matches table using a Query

In [196]:
responce = requests.get('https://api.opendota.com/api/explorer?sql=SELECT%20%2A%20FROM%20matches%20LIMIT%201%20ORDER%20BY%20%27start_time%27%3B')

In [197]:
responce

<Response [400]>

In [167]:
matches = responce.json()

In [169]:
len(matches)

11

In [171]:
matches.keys()

dict_keys(['command', 'rowCount', 'oid', 'rows', 'fields', '_parsers', '_types', 'RowCtor', 'rowAsArray', '_prebuiltEmptyResultObject', 'err'])

Mappings for team 0 (radiant) team 1 is Dire

In [138]:
matches['rows'][0].keys()

dict_keys(['match_id', 'match_seq_num', 'radiant_win', 'start_time', 'duration', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire', 'cluster', 'first_blood_time', 'lobby_type', 'human_players', 'leagueid', 'positive_votes', 'negative_votes', 'game_mode', 'engine', 'picks_bans', 'radiant_team_id', 'dire_team_id', 'radiant_team_name', 'dire_team_name', 'radiant_team_complete', 'dire_team_complete', 'radiant_captain', 'dire_captain', 'chat', 'objectives', 'radiant_gold_adv', 'radiant_xp_adv', 'teamfights', 'version', 'cosmetics', 'radiant_score', 'dire_score', 'draft_timings', 'series_id', 'series_type', 'replay_salt'])

In [178]:
matches['rows'][0]['match_id']

7502154550

In [214]:
responce = requests.get('https://api.opendota.com/api/explorer?sql=SELECT%20match_id%2C%20start_time%20FROM%20matches%20ORDER%20BY%20match_id%20DESC%20LIMIT%201000%3B')

In [216]:
responce

<Response [200]>

In [218]:
matches = responce.json()

# duration given in seconds

In [5]:
def create_match_table():
    conn = psql.connect(database = 'pagila',
                    user = os.getenv("sql_user"),
                    host = os.getenv("host"),
                    password = os.getenv("sql_password"),
                    port=5432
                    )
    cur = conn.cursor()
    creation_slq = '''
            CREATE TABLE IF NOT EXISTS student.ojdb_matches (
                match_id BIGINT PRIMARY KEY,
                rank SMALLINT,
                radiant_wins BOOLEAN NOT NULL
            );
    '''
    # run query
    cur.execute(creation_slq)
    conn.commit()
    # close connections
    cur.close()
    conn.close()


In [7]:
def create_hero_picks_table():
    conn = psql.connect(database = 'pagila',
                    user = os.getenv("sql_user"),
                    host = os.getenv("host"),
                    password = os.getenv("sql_password"),
                    port=5432
                    )
    cur = conn.cursor()
    creation_slq = '''
            CREATE TABLE IF NOT EXISTS student.ojdb_hero_picks (
                match_id BIGINT REFERENCES student.ojdb_matches(match_id),
                hero_id SMALLINT,
                team SMALLINT,
                facet SMALLINT,
                items SMALLINT[] NOT NULL CHECK (array_length(items, 1) = 6),
                backpack SMALLINT[] NOT NULL CHECK (array_length(backpack, 1) = 3),
                neutral_item SMALLINT,
                kills SMALLINT,
                deaths SMALLINT,
                assists SMALLINT,
                gold_per_min SMALLINT,
                xp_per_min SMALLINT,
                level SMALLINT,
                net_worth INT,
                aghanims_scepter BOOLEAN,
                aghanims_shard BOOLEAN,
                moonshard BOOLEAN,
                hero_damage INT,
                tower_damage INT,
                hero_healing INT,
                PRIMARY KEY(match_id, hero_id)
            );
    '''
    cur.execute(creation_slq)
    conn.commit()
    # close connections
    cur.close()
    conn.close()

In [49]:
create_match_table()

In [51]:
create_hero_picks_table()

In [48]:
def get_matches():
    create_match_table()
    create_hero_picks_table()
    rank_distributions = ((10,15), # herald
                          (20,25), # guardian
                          (30,35), # crusader
                          (40,45), # archon
                          (50,55), # legend
                          (60,65), # ancient
                          (70,75), # divine
                          (80,85)) # immortal
    
    with psql.connect(database = 'pagila',
                user = os.getenv("sql_user"),
                host = os.getenv("host"),
                password = os.getenv("sql_password"),
                port=5432) as conn:
        with conn.cursor() as cur:
            for rank_dist in rank_distributions:
                matches = requests.get(
                    f"https://api.opendota.com/api/publicMatches?min_rank={rank_dist[0]}&max_rank={rank_dist[1]}"
                ).json()
                insert_match_query = """
                                    INSERT INTO student.ojdb_matches (match_id, avg_rank, radiant_wins)
                                    VALUES (%s, %s, %s)
                                    ON CONFLICT (match_id) DO NOTHING;
                                    """
                insert_player_query = """
                                    INSERT INTO student.ojdb_hero_picks (match_id, hero_id, team, facet, items, backpack, neutral_item,
                                    kills, deaths, assists, gold_per_min, xp_per_min, level, net_worth, aghanims_scepter, aghanims_shard,
                                    moonshard, hero_damage, tower_damage, hero_healing)
                                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                                    ON CONFLICT (match_id, hero_id) DO NOTHING;
                                    """
                
                for match in matches:
                    match_values = (
                        match['match_id'],
                        match['avg_rank_tier'],
                        match['radiant_win']           
                        )
                    cur.execute(insert_match_query, match_values)
                    match_data = requests.get(
                        f"https://api.opendota.com/api/matches/{match['match_id']}"
                    ).json()
                    player_data = match_data['players']
                    for player in player_data:
                        player_values = (
                            match['match_id'],
                            player['hero_id'],
                            player['team_number'],
                            player['hero_variant'],
                            [player[f'item_{i}'] for i in range(6)],
                            [player[f'backpack_{i}'] for i in range(3)],
                            player['item_neutral'],
                            player['kills'],
                            player['deaths'],
                            player['assists'],
                            player['gold_per_min'],
                            player['xp_per_min'],
                            player['level'],
                            player['net_worth'],
                            bool(player['aghanims_scepter']),
                            bool(player['aghanims_shard']),
                            bool(player['moonshard']),
                            player['hero_damage'],
                            player['tower_damage'],
                            player['hero_healing']
                        )
                        cur.execute(insert_player_query, player_values)
                    conn.commit()
                
        
        
    

In [50]:
get_matches()

In [98]:
responce = requests.get('https://api.opendota.com/api/publicmatches')

In [100]:
responce

<Response [200]>

In [102]:
matches = responce.json()

In [104]:
matches[0]

{'match_id': 7817650804,
 'match_seq_num': 6580202549,
 'radiant_win': True,
 'start_time': 1719498044,
 'duration': 657,
 'lobby_type': 0,
 'game_mode': 22,
 'avg_rank_tier': 33,
 'num_rank_tier': 6,
 'cluster': 225,
 'radiant_team': [31, 2, 94, 7, 56],
 'dire_team': [109, 101, 17, 41, 81]}

In [72]:
responce = requests.get('https://api.opendota.com/api/matches/7817317900')

In [22]:
create_match_table()

In [74]:
responce

<Response [200]>

In [76]:
match_data = responce.json()

In [78]:
match_data.keys()

dict_keys(['players', 'radiant_win', 'duration', 'pre_game_duration', 'start_time', 'match_id', 'match_seq_num', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire', 'cluster', 'first_blood_time', 'lobby_type', 'human_players', 'leagueid', 'game_mode', 'flags', 'engine', 'radiant_score', 'dire_score', 'picks_bans', 'od_data', 'metadata', 'patch', 'region'])

In [108]:
match_data['players'][0].keys()

dict_keys(['player_slot', 'team_number', 'team_slot', 'hero_id', 'hero_variant', 'item_0', 'item_1', 'item_2', 'item_3', 'item_4', 'item_5', 'backpack_0', 'backpack_1', 'backpack_2', 'item_neutral', 'kills', 'deaths', 'assists', 'leaver_status', 'last_hits', 'denies', 'gold_per_min', 'xp_per_min', 'level', 'net_worth', 'aghanims_scepter', 'aghanims_shard', 'moonshard', 'hero_damage', 'tower_damage', 'hero_healing', 'gold', 'gold_spent', 'ability_upgrades_arr', 'radiant_win', 'start_time', 'duration', 'cluster', 'lobby_type', 'game_mode', 'is_contributor', 'patch', 'region', 'isRadiant', 'win', 'lose', 'total_gold', 'total_xp', 'kills_per_min', 'kda', 'abandons', 'rank_tier', 'is_subscriber', 'benchmarks'])